In [1]:
import cv2
import os
import numpy as np
from scipy.spatial import KDTree
import pickle
from random import sample
import random
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [2]:
"""step1:对视频进行处理，得到每5s的视频段sift特征"""
"""对图片进行SIFT特征提取"""
def compute_sift(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # sift特征提取
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=30)
    kp, des = sift.detectAndCompute(gray, None)  # keypoints, descriptors
    return des

"""处理视频"""
def detect_sift(video_input_path,box_coordinate_path,store_path):
    video = cv2.VideoCapture(video_input_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_index = 0
    # 对产生的坐标信息进行保存
    feature_clip = []
    # 打开坐标文件
    with open(box_coordinate_path,"r") as f:
            line1 = f.readline().strip().replace("[","")
            line1 = line1.replace("]","")
            line1 = line1.split(",")
            box_coordinate = list(map(int,line1))
    while(video.isOpened()):
        ret, frame = video.read()   # 读取视频帧
        if ret == True:
            # 根据box大小剪裁视频
            frame = frame[box_coordinate[1]:box_coordinate[3],box_coordinate[0]:box_coordinate[2]]
            des = compute_sift(frame)   # sift特征计算
            if des is not None:
                feature_clip.append(des)
            # 保存每段clip的全部帧的sift特征
            frame_index += 1
        else:
            break
        ch = 0xFF & cv2.waitKey(1)
        if ch == 27:
            break
    feature_clip_array = np.array(feature_clip)
    with open(store_path,"wb") as file:
        pickle.dump(feature_clip_array,file)

In [24]:
videos_dir = "./裁剪后视频段"
boxes_dir = "./检测框信息"
stores_dir = "./sift原始特征"
for video_name in os.listdir(videos_dir):
    video_path = os.path.join(videos_dir,video_name)
    box_path = os.path.join(boxes_dir,video_name+".txt")
    store_path = os.path.join(stores_dir,video_name)
    # 如果路径不存在则建立文件夹
    if not os.path.exists(store_path):
        os.makedirs(store_path)
    for clip_name in os.listdir(video_path):
        tmp_name = clip_name.split('.')[0]+".pkl"
        clip_path = os.path.join(video_path,clip_name)
        clip_store_path = os.path.join(store_path,tmp_name)
        detect_sift(clip_path,box_path,clip_store_path)

C:\Users\admin\AppData\Local\Temp/ipykernel_30372/2669141145.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  feature_clip_array = np.array(feature_clip)


In [3]:
"""step2：构建字典，构建向量"""
"""统计所有视频的sift向量，构建并训练bow词袋模型"""
def create_dictionary(features):
    dictionary_size=20
    bow = cv2.BOWKMeansTrainer(dictionary_size)
    # 将所有视频的所有clip添加到bow里
    for clip_idx, clip in enumerate(features):
        for j, frame in enumerate(clip):
            bow.add(frame)
    # 聚类
    dictionary = bow.cluster()
    np.savetxt("sift_bow_dict.txt",dictionary)
    return dictionary

"""重构词向量"""
def create_bow(dictionary, features, store_paths, method):
    kdtree = KDTree(dictionary)
    bins = dictionary.shape[0]
    for clip_idx, clip in enumerate(features):
        clip_list = []
        for j, frame in enumerate(clip):
            dis, indices = kdtree.query(frame)
            hist, _ = np.histogram(indices, bins=bins)
            hist = hist / len(indices)
            clip_list.append(hist)
        clip_list = np.array(clip_list)

        if method == "mean":
            clip_list = np.mean(clip_list, 0)
        elif method == "max":
            clip_list = np.max(clip_list, 0)
        else:
            raise Exception('Method not found')
        np.save(store_paths[clip_idx], clip_list)

In [18]:
"""主函数：处理视频并保存每个clip的bow向量"""
def process():
    # 相关文件路径
    videos_sift_dir = "./sift原始特征"
    stores_dir = "./sift统计"
    
    features = []
    stores_path = []
    # 将所有初始sift特征从文件中提取出来
    for filename in sorted(os.listdir(videos_sift_dir)):
        video_sift_path = os.path.join(videos_sift_dir,filename)
        store_path = os.path.join(stores_dir,filename)
        if not os.path.exists(store_path):
            os.makedirs(store_path)
        for clipname in sorted(os.listdir(video_sift_path)):
            clip_sift_path = os.path.join(video_sift_path,clipname)
            clip_store_path = os.path.join(store_path,clipname.split(".")[0])
            stores_path.append(clip_store_path)
            with open(clip_sift_path,"rb") as file:
                clip_feature = pickle.load(file)
                features.append(clip_feature)
    print("Done: features创建完成")
    
    # 建立词典--分配词向量
    dictionary = create_dictionary(features)
    print("Done: dictionary 创建完成")
    create_bow(dictionary, features, stores_path, "mean")
    print("Done: bow创建完成")

In [19]:
process()

(30, 128)
Done: features创建完成


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\bagofwords.cpp:94: error: (-215:Assertion failed) !descriptors.empty() in function 'cv::BOWKMeansTrainer::cluster'


In [4]:
"""读取保存的bow词特征向量"""
def read_bow():
    bow_feature_path = "./sift统计"
    labels_path = "./标签信息"
    feature_sample = [[],[],[],[]]
    filename_sample = [[],[],[],[]]
    for videoname in sorted(os.listdir(bow_feature_path)):
        video_feature_path = os.path.join(bow_feature_path,videoname)   # 视频路径
        label_path = os.path.join(labels_path,videoname+".txt")    # 标签路径
        # 读取标签文件
        label = []
        with open(label_path,"r",encoding='utf-8') as f:
            while True:
                line = f.readline()
                if not line:
                    break
                line = line.split('，')
                label.append(eval(line[1]))
        # 根据label读取每个clip的特征文件
        for i in range(len(label)):
            clipname = "clip"+str(i+1)+".npy"
            tmp_path = videoname+"/"+clipname
            clip_feature_path = os.path.join(video_feature_path,clipname)
            feature = np.load(clip_feature_path)   # 读取feature文件
            feature_sample[label[i]-1].append(feature.tolist())
            filename_sample[label[i]-1].append(tmp_path)
    return feature_sample, filename_sample

In [7]:
"""划分数据集并利用svm，knn训练"""
def train_svm_knn(clf_num,method):
    feature_sample, filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        if method == "SVM":
#             clf = svm.SVC(kernel='linear',decision_function_shape='ovr',max_iter=1e7)
            clf = svm.SVC(kernel='linear',decision_function_shape='ovr')
            clf.fit(train_data,train_label)
        if method == "KNN":
            clf = KNeighborsClassifier(n_neighbors=10)
            clf.fit(train_data,train_label)
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
#         print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
#         print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print("Method:",method," Clf_num",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [8]:
"""进行2，3，4分类，并利用svm，knn为分类器做对比实验"""
cls_num = [2,3,4]
methods = ["SVM","KNN"]
for num in cls_num:
    for method in methods:
        train_svm_knn(num,method)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: SVM  Clf_num 2
训练集准确率： 0.6883667180277351
测试集准确率： 0.5651826484018264
精确率： 0.566938468574259
召回率： 0.5651826484018264
F1得分： 0.5553218774586596
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: KNN  Clf_num 2
训练集准确率： 0.8399514045276758
测试集准确率： 0.7146499238964992
精确率： 0.7214755424830435
召回率： 0.7146499238964992
F1得分： 0.7112244927189598
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: SVM  Clf_num 3
训练集准确率： 0.5942174879384181
测试集准确率： 0.4438386524822695
精确率： 0.4326307055357269
召回率： 0.4438386524822695
F1得分： 0.43041958821005427
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: KNN  Clf_num 3
训练集准确率： 0.7302564102564102
测试集准确率： 0.5216312056737589
精确率： 0.5374786318506664
召回率： 0.5216312056737589
F1得分： 0.5

In [13]:
"""利用mlp、rvm、random forest分类器进行分类实验"""
"""补充实验"""
def train_supplement(clf_num,method):
    feature_sample, filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        if method == "MLP":
            # 利用mlp进行多分类
            from sklearn.neural_network import MLPClassifier
#             clf = MLPClassifier(solver='lbfgs',alpha=1e-6,hidden_layer_sizes=(64,3))
#             clf = MLPClassifier(solver='lbfgs',alpha=1e-6,hidden_layer_sizes=(128,6))
            clf = MLPClassifier(solver='lbfgs',alpha=1e-6,hidden_layer_sizes=(256,9))
            clf.fit(train_data,train_label)
        if method == "RVM":
            from sklearn_rvm import EMRVC
            clf = EMRVC(kernel='linear')
            clf.fit(train_data,train_label)
        if method == "RandomForest":
            from sklearn.ensemble import RandomForestClassifier
            clf = RandomForestClassifier(max_depth=3)
            clf.fit(train_data,train_label)
        if method == "RVM":
            from sklearn_rvm import EMRVC
            clf = EMRVC(kernel='rbf',max_iter=10)
            clf.fit(train_data,train_label)
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
        print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
        print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print("Method:",method," Clf_num:",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [14]:
cls_num = [2,3,4]
methods = ["RVM"]
for num in cls_num:
    for method in methods:
        train_supplement(num,method)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "


当前折训练集准确率： 0.9722650231124808
当前折测试集准确率： 0.6712328767123288


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "


当前折训练集准确率： 0.9799691833590138
当前折测试集准确率： 0.684931506849315


...

当前折训练集准确率： 0.9546666666666667
当前折测试集准确率： 0.5365853658536586


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will c

当前折训练集准确率： 0.96
当前折测试集准确率： 0.5121951219512195


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will c

当前折训练集准确率： 0.96
当前折测试集准确率： 0.4146341463414634
Method: RVM  Clf_num: 4
训练集准确率： 0.9580670231729055
测试集准确率： 0.47595818815331004
精确率： 0.4934140404366117
召回率： 0.47595818815331004
F1得分： 0.4707782943752651


In [9]:
"""划分数据集并利用dt训练"""
def train_dt(clf_num):
    feature_sample, filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        
        clf = DecisionTreeClassifier(random_state=0,max_depth=3)
        clf.fit(train_data,train_label)
    
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
#         print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
#         print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print(" Clf_num",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [10]:
"""进行2，3，4分类，并利用dt为分类器做对比实验"""
cls_num = [2,3,4]
for num in cls_num:
    train_dt(num)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
 Clf_num 2
训练集准确率： 0.742691952115681
测试集准确率： 0.6830289193302892
精确率： 0.6842777700701845
召回率： 0.6830289193302892
F1得分： 0.6825378033679025
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
 Clf_num 3
训练集准确率： 0.5613807123109449
测试集准确率： 0.4441932624113475
精确率： 0.4407710538324934
召回率： 0.4441932624113475
F1得分： 0.40796698351996047


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
 Clf_num 4
训练集准确率： 0.45432299465240644
测试集准确率： 0.31742160278745646
精确率： 0.36763168645326
召回率： 0.31742160278745646
F1得分： 0.27646680398925566


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
